<a href="https://colab.research.google.com/github/connectalizain/LangChain_HelloWorld/blob/main/langChain_Pinecone_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install -qU langchain-pinecone langchain-google-genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 41.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 66.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 433.9/433.9 kB 31.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 421.9/421.9 kB 32.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.2/52.2 kB 4.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-generativeai 0.8.4 requires google-ai-generativelanguage==0.6.15, but you have google-ai-generativelanguage 0.6.17 which is incompatible.


In [ ]:
from google.colab import userdata
userdata.get('PINECONE_API_KEY')
from pinecone import Pinecone, ServerlessSpec

pinecone_api_key = userdata.get('PINECONE_API_KEY')

pc = Pinecone(api_key=pinecone_api_key)

In [ ]:

index_name = "my-firstt-rag"
pc.create_index(
        name=index_name,
        dimension=768,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
    )

index = pc.Index(index_name)


*   768 is code from Google
*   512/384 are other options but they use less data




In [ ]:
import os
from langchain_google_genai import GoogleGenerativeAIEmbeddings


os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')
embeddings = GoogleGenerativeAIEmbeddings(model="models/text-embedding-004")
vector = embeddings.embed_query("hello, world!")
vector[:5]

[0.014134909026324749,
 -0.022324152290821075,
 -0.054603420197963715,
 -0.006284549366682768,
 -0.03392402455210686]

In [ ]:
from langchain_pinecone import PineconeVectorStore

vector_store = PineconeVectorStore(index=index, embedding=embeddings)

In [ ]:
from uuid import uuid4

from langchain_core.documents import Document

document_1 = Document(
    page_content=" Owl,Whispers float through midnight air, Wisdom wrapped in a feathery glare.",
    metadata={"source": "gpt"},
)

document_2 = Document(
    page_content="Cat, Silent steps and secret stares, Queen of calm and cozy lairs",
    metadata={"source": "gpt"},
)

document_3 = Document(
    page_content="🐘 Elephant, With gentle grace and giant might, She walks like thunder, calm and bright.",
    metadata={"source": "gpt"},
)

document_4 = Document(
    page_content="Robbers broke into the city bank and stole $1 million in cash.",
    metadata={"source": "news"},
)

document_5 = Document(
    page_content="Wow! That was an amazing movie. I can't wait to see it again.",
    metadata={"source": "tweet"},
)

document_6 = Document(
    page_content="Is the new iPhone worth the price? Read this review to find out.",
    metadata={"source": "website"},
)

document_7 = Document(
    page_content="The top 10 soccer players in the world right now.",
    metadata={"source": "website"},
)

document_8 = Document(
    page_content="LangGraph is the best framework for building stateful, agentic applications!",
    metadata={"source": "tweet"},
)

document_9 = Document(
    page_content="The stock market is down 500 points today due to fears of a recession.",
    metadata={"source": "news"},
)

document_10 = Document(
    page_content="I have a bad feeling I am going to get deleted :(",
    metadata={"source": "tweet"},
)

documents = [
    document_1,
    document_2,
    document_3,
    document_4,
    document_5,
    document_6,
    document_7,
    document_8,
    document_9,
    document_10,
]
uuids = [str(uuid4()) for _ in range(len(documents))]

vector_store.add_documents(documents=documents, ids=uuids)

['a64a23e7-5c23-4523-bcbf-5fe4cd4a9ea1',
 '27f8ecf9-a123-41ae-bb36-e6aedc5d9cba',
 '402eef35-e700-4668-9d84-37835e71de3c',
 'c0158f34-5004-4bff-a5ab-5ec61610e17c',
 'ef41810f-b4d7-4ea5-80e3-b0c56a3c6bae',
 'b91f2374-49b2-4974-89d1-485b17376360',
 '831d4fa6-1d09-426a-b8f7-95ca754618be',
 'a348b5b5-5f14-4b96-be70-5ab60d489e04',
 '124548e3-b880-4d7e-8449-b250e399c27f',
 '9ae60998-a5f5-421f-a0e0-7276a6b4b9dd']

QUERY SEARCH WITH SIMILARITY


*   K is used for number of answers you want
*   FILTERS are used search data using key data pair


  



In [ ]:
results = vector_store.similarity_search(
    "poetry",
    # k=2,
    filter={"source": "gpt"},
)
for res in results:
    print(f"* {res.page_content} [{res.metadata}]")

*  Owl,Whispers float through midnight air, Wisdom wrapped in a feathery glare. [{'source': 'gpt'}]
* Dog, Loyal eyes and wagging tail, A heart so big, it’ll never fail. [{'source': 'gpt'}]
* Dog, Loyal eyes and wagging tail, A heart so big, it’ll never fail. [{'source': 'gpt'}]
* Dog, Loyal eyes and wagging tail, A heart so big, it’ll never fail. [{'source': 'gpt'}]


SIMILARITY SEARCH WITH SCORE

In [ ]:
results = vector_store.similarity_search_with_score(
    "Will it be hot tomorrow?", k=2, filter={"source": "news"}
)
for res, score in results:
    print(f"* [SIM={score:3f}] {res.page_content} [{res.metadata}]")

* [SIM=0.639507] The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees. [{'source': 'news'}]
* [SIM=0.414024] The stock market is down 500 points today due to fears of a recession. [{'source': 'news'}]


In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model= "gemini-1.5-flash",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...
)

In [ ]:
messages = [
    (
        "system",
        "You are a helpful assistant that translates English to French. Translate the user sentence.",
    ),
    ("human", "I love programming."),
]
ai_msg = llm.invoke(messages)
ai_msg
print(f"your result: {ai_msg.content} ")

your result: That's great!  Programming is a rewarding and challenging field. What aspects of programming do you enjoy the most? 


In [ ]:
def answer_to_user(query: str):
  vector_results = vector_store.similarity_search( query,
    k=2)
  final_ans = llm.invoke(f"please share more info regarding this {vector_results}")
  print(final_ans.content)

In [ ]:
answer_to_user("elobrate this poetry about owl")

This data represents a list of two documents generated by GPT (likely a large language model like GPT-3 or GPT-4).  Let's break down each document:

**Document 1:**

* **`id='a64a23e7-5c23-4523-bcbf-5fe4cd4a9ea1'`**: This is a unique identifier for the document.  It's a universally unique identifier (UUID), ensuring that this document can be distinguished from any other.

* **`metadata={'source': 'gpt'}`**: This metadata indicates the origin of the document.  `'gpt'` signifies that it was generated by a GPT model.

* **`page_content=' Owl,Whispers float through midnight air, Wisdom wrapped in a feathery glare.'`**: This is the actual content of the document – a short, poetic description of an owl.  It uses imagery and metaphor to evoke a sense of mystery and wisdom.


**Document 2:**

* **`id='ad292e01-9b03-454a-80de-cbfd763a459b'`**:  Another unique identifier for this document.

* **`metadata={'source': 'gpt'}`**:  Again, indicating GPT as the source.

* **`page_content='🐘 Elephant, 